**Data Science Challenge: Card Transactions!**
This coding and analysis challenge is designed to test your skill and intuition analyzing real[-ish] world data. For the challenge, we will use credit card transactions data. Note that this dataset loosely resembles real transactional data from Capital One credit card customers, but the entities and relations within are purely fictional. No persons, places, or things lost their identity in the making of this dataset.

**Required Questions:** Please answer completely all four required questions.


**Question 1:** Load\
- Programmatically download and load into your favorite analytical tool the transactions data. This data, which is in line-delimited JSON format, can be found here

- Please describe the structure of the data. Number of records and fields in each record?

- Please provide some additional basic summary statistics for each field. Be sure to include a count of null, minimum, maximum, and unique values where appropriate.

**Question 2:** Plot\
- Plot a histogram of the processed amounts of each transaction, the transactionAmount column.

- Report any structure you find and any hypotheses you have about that structure.

**Question 3:** Data Wrangling - Duplicate Transactions\
You will notice a number of what look like duplicated transactions in the data set. One type of duplicated transaction is a reversed transaction, where a purchase is followed by a reversal. Another example is a multi-swipe, where a vendor accidentally charges a customer's card multiple times within a short time span.

- Can you programmatically identify reversed and multi-swipe transactions?

- What total number of transactions and total dollar amount do you estimate for the reversed transactions? For the multi-swipe transactions? (please consider the first transaction to be "normal" and exclude it from the number of transaction and dollar amount counts)

- Did you find anything interesting about either kind of transaction?

**Question 4:** Model\
Fraud is a problem for any bank. Fraud can take many forms, whether it is someone stealing a single credit card, to large batches of stolen credit card numbers being used on the web, or even a mass compromise of credit card numbers stolen from a merchant via tools like credit card skimming devices.

- Each of the transactions in the dataset has a field called isFraud. Please build a predictive model to determine whether a given transaction will be fraudulent or not. Use as much of the data as you like (or all of it).

- Provide an estimate of performance using an appropriate sample, and show your work.

- Please explain your methodology (modeling algorithm/method used and why, what features/data you found useful, what questions you have, and what you would do next with more time)

# Download the transaction file from github

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from imblearn.over_sampling import SMOTE
pd.set_option('display.max_columns', None)

In [ ]:
#df = pd.read_json("https://github.com/CapitalOneRecruiting/DS/raw/master/transactions.zip", lines=True )
# df.to_csv("transaction.csv", index=False)
df = pd.read_csv("transaction.csv",infer_datetime_format=True, encoding_errors='ignore')

In [ ]:
df.tail(10)

In [ ]:
df.transactionType.unique()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
print("The number of Row:", df.shape[0])
print("The number of columns:", df.shape[1])

In [ ]:
df.describe(include='all')

In [ ]:
#df_copy.isnull().sum()

- There is an indication of invalid entries in columns such as `echoBuffer`,`merchantCity`,`merchantState`,`merchantZip`,`posOnPremises`,`recurringAuthInd`
- The columns listed above contain empty string values



In [ ]:
df['transactionAmount'].plot(kind='hist',figsize = (12,8),title="Histogram of the Processed Amounts of each Transaction");

- It is observe that the distribution of the processed amount of each transaction is not normally distributed, all the transactions are center around 0 to 250

# Data Wragling 

In [ ]:
# replace empty strings with NaN
df.replace('', np.nan, inplace=True)

- **The original data type of transactionDateTime is an object, it needs to be converted to datetime.** 

In [ ]:
df["transactionDateTime"]=pd.to_datetime(df["transactionDateTime"]) # convert to appropriate data type

In [ ]:
df.info() # check

In [ ]:
# get records of reversed transactions
reversed_df = df[df['transactionType']=='REVERSAL']
print(f'The total number of reversed transactions is {reversed_df.shape[0]}')
print(f'while the total dollar amount estimated for the reversed transactions is ${reversed_df["transactionAmount"].sum()}')

In [ ]:
# col_of_interest = ['accountNumber',"customerId", 'transactionDateTime', 'cardLast4Digits', 'cardCVV',
#                    'transactionAmount', 'transactionType', "availableMoney","merchantName",
#                    "posEntryMode","posConditionCode","merchantCategoryCode",
#                    "currentBalance"]
sorted_df = df.sort_values(by=['accountNumber',"customerId", 'cardLast4Digits','transactionDateTime', "transactionAmount"])

# define a function to identify multiple swipes

def identify_multiple_swipes(sub_df, time_window='2min'):
    sub_df['transactionDateTime'] = pd.to_datetime(sub_df['transactionDateTime'])
    sub_df['transactionDateTimeDiff'] = (sub_df['transactionDateTime'].shift(-1) - sub_df['transactionDateTime'])

    sub_df['isMultipleSwipe'] = (
        (sub_df['accountNumber'].eq(sub_df['accountNumber'].shift(-1))) &
        (sub_df['customerId'].eq(sub_df['customerId'].shift(-1))) &
        (sub_df['cardLast4Digits'].eq(sub_df['cardLast4Digits'].shift(-1))) &
        (sub_df['transactionAmount'].eq(sub_df['transactionAmount'].shift(-1))) &
        (sub_df['transactionType'].shift(-1)!= "REVERSAL") &
        ((sub_df['transactionDateTime'].shift(-1) - sub_df['transactionDateTime']) <= pd.to_timedelta(time_window))
    )
   
    return sub_df

# Apply the function to the DataFrame
sorted_df_new = identify_multiple_swipes(sorted_df)

In [ ]:
sorted_df_new

In [ ]:
# filter to get only those transactions flagged as multiple swipes
multiple_swipes_df = sorted_df_new[sorted_df_new['isMultipleSwipe']]
# show
multiple_swipes_df

In [ ]:
# manually confirm
sorted_df_new[(sorted_df_new['accountNumber']==100737756) &
                (sorted_df_new['customerId']==100737756) & 
                (sorted_df_new['cardLast4Digits']==4317) & 
                (sorted_df_new['transactionAmount']==693.5)]

In [ ]:
print(f'The total number of multiple swipe transactions is {multiple_swipes_df.shape[0]}')
print(f'while the total dollar amount estimated for the multiple swipe transactions is ${multiple_swipes_df["transactionAmount"].sum()}')

# Building The Model

### Data Cleaning and Exploration

In [ ]:
# make a copy of the dataframe for cleaning
data = sorted_df_new.copy()

In [ ]:
# drop duplicated records(reversed and multiple swipe transactions)
data['transactionType'] = data['transactionType'].replace("REVERSAL", np.nan)
data['isMultipleSwipe'] = data['isMultipleSwipe'].replace(True, np.nan)
data.dropna(subset=['transactionType', 'isMultipleSwipe'], inplace=True)

In [ ]:
data['isMultipleSwipe'].unique() # check that isMultipleSwipe is only False

In [ ]:
#confirm there are no missing values in transactionType
data.isna().sum()

**Looking at the missing value report above, it could be seen there are columns with 100% missing values: It will be ideal to remove such columns totally from the data:**

In [ ]:
# calculate percentage missing value
pct_missing_before = ((data.isnull().sum()/data.isnull().count() * 100).sort_values(ascending=False)).round(2)
print("BEFORE DROPPING COLUMNS WITH 100% MISSING VALUES: \n",pct_missing_before)

# remove the columns with 100% missing values
data.drop(columns=['echoBuffer', 'merchantCity', 'merchantState', 'merchantZip', 'posOnPremises', 'recurringAuthInd'], 
         inplace=True)

# calculate percentage of missing value after droping the columns with 100% missing data 
pct_missing_after = ((data.isnull().sum()/data.isnull().count() * 100).sort_values(ascending=False)).round(2)
print("\nBEFORE DROPPING COLUMNS WITH 100% MISSING VALUES: \n",pct_missing_after)

Now that we have dropped the duplicated records and the columns with 100% missing values from the dataset, let's take a quick exploration on the structure of the cleaned data:

In [ ]:
data.shape

In [ ]:
# check for the distribution of the target class
data['isFraud'].value_counts()

This indicates an imbalanced data. Let's visualize this distribution to get this better:

In [ ]:
plt.figure(figsize = (10,8))
sns.countplot(x='isFraud', data=df);
plt.title("Target Variable Distribution")
plt.show();

It could be confirmed from the plot above that we have an imbalanced data which we will have to consider and handle in our model.



Looking at the independent variables, it will be ideal excluding certain columns that might not provide useful information or could potentially lead to data leakage. **Here are some columns I consider excluding:**

- `accountNumber`, `customerId`: These are identifiers and may not contribute to fraud prediction.

- `cardCVV`, `enteredCVV`, `cardLast4Digits`: Features related to the card itself might not be very informative directly but we can derive a new feature that tells whether the cardCVV matches with the enteredCVV during purchase - `matchingCVV`.

- `availableMoney`, `currentBalance`: These columns might not be available during the time of the transaction or could directly correlate with the `transactionAmount`, potentially causing **data leakage**.

- `transactionDateTime`: While the timing of transactions is important, it's generally not a good idea to include the exact timestamp directly. We'll derive features such as `transactionHour`, `transactionDayOfWeek`, `transactionMonth`, and `transactionDayOfMonth`.

- `merchantName`: This column may not be directly related to fraud and might introduce noise - since the number of merchant name that can exist is infinite. However if time permits, we might consider engineering features from it, such as the number of transactions from a specific merchant.

- `currentExpDate`, `accountOpenDate`, `dateOfLastAddressChange`, `expirationDateKeyInMatch`: These date-related columns might not directly contribute to fraud prediction. but we can derive features such as the age of the account (`ageOfAccount`) or time since the last address change (`timeSinceLastAddressChange`).



We'll see from the remaining variables the ones that have a unique interaction with the target variable to further justify our bases for selecting them for modeling.

However, before we check for interaction, let's derive some new variables from existing variables, that could be of help to our model. These variables will be `transactionDayOfWeek`, `transactionMonth`, `transactionDayOfMonth`, `transactionHour`, `ageOfAccount`, `timeSinceLastAddressChange` and also, `matchingCVV` which tells if the cardCVV matches with the enteredCVV during purchase. 

In [ ]:

# extract time related features
data['transactionDayOfWeek'] = data['transactionDateTime'].dt.day_name()
data['transactionMonth'] = data['transactionDateTime'].dt.month_name()
data['transactionDayOfMonth'] = data['transactionDateTime'].dt.day
data['transactionHour'] = data['transactionDateTime'].dt.hour

data["accountOpenDate"]=pd.to_datetime(data["accountOpenDate"]) # convert to appropriate data type
data["dateOfLastAddressChange"]=pd.to_datetime(data["dateOfLastAddressChange"]) # convert to appropriate data type
data['ageOfAccount'] = (data['transactionDateTime'] - data['accountOpenDate']).dt.days
data['timeSinceLastAddressChange'] = (data['transactionDateTime'] - data['dateOfLastAddressChange']).dt.days

# get matchingCVV
data['matchingCVV'] = (data['cardCVV'] ==data['enteredCVV'])

In [ ]:
# drop columns not of interest
cols_not_of_interest = ['accountNumber', 'customerId', 'transactionDateTime', 'cardCVV', 'enteredCVV', 'cardLast4Digits',
                        'accountOpenDate', 'dateOfLastAddressChange', 'transactionDateTimeDiff', 'isMultipleSwipe',
                       'merchantName', 'currentExpDate', 'availableMoney', 'currentBalance'
                        ]
data.drop(columns=cols_not_of_interest, inplace=True)

In [ ]:
# let's check the missing values proper
data.isna().sum()

I will recommend uniquely identifying the missing values in these columns as they might not be missing at random. Categorical 

In [ ]:
data['acqCountry'] = data['acqCountry'].replace(np.nan, 'MISSING')
data['merchantCountryCode'] = data['merchantCountryCode'].replace(np.nan, 'MISSING')
data['posEntryMode'] = data['posEntryMode'].replace(np.nan, '99')
data['posConditionCode'] = data['posConditionCode'].replace(np.nan, '99')

In [ ]:
# identify categorical and continuous independent variables, as well as the target variable:
num_cols = ['creditLimit', 'transactionAmount', 'ageOfAccount','timeSinceLastAddressChange', 'transactionDayOfMonth']
cat_cols = ['acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode',
            'merchantCategoryCode', 'transactionType', 'cardPresent',
            'expirationDateKeyInMatch', 'transactionMonth', 'transactionDayOfWeek',
            'transactionHour', 'matchingCVV']

label_class = 'isFraud'


fig = plt.figure(figsize=(20,50))

for i, col in enumerate(cat_cols):
    ax1 = fig.add_subplot(6, 2, i+1)
    x, y = col, "proportion"
    (data[x]
     .groupby(data[label_class])
     .value_counts(normalize=True)
     .rename(y)
     .reset_index()
     .pipe((sns.barplot, "data"), x=x, y=y, hue=label_class, ax=ax1))
plt.title("Proportion plots for categorical independent variables against dependent variable ")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Some of the insights here are that:
- We can tell transactions with card not present are more likely to be fraudulent
- transaction with posEntryMode of 09, 02, 90 and 99 (missing) are more likely to be fraudulent while mode 05 is more likely legitimate transaction
- Also missing posConditionMode (99) are more likely to be fraudulent
- When there's non-matching CVV, the transaction has tendencies of being fraudulent


Hence, the categorical variables that will be considered as a feature for the model are:
- 'acqCountry'
- 'merchantCountryCode'
- 'posEntryMode'
- 'posConditionCode',
- 'merchantCategoryCode'
- 'transactionType'
- 'cardPresent'
- 'transactionMonth'
- 'transactionDayOfWeek',
- 'transactionHour'
- 'matchingCVV'

In [ ]:
fig = plt.figure(figsize=(6,40))

for i, col in enumerate(num_cols):
    ax1 = fig.add_subplot(5, 1, i+1)
    sns.boxplot(x=label_class, y=col, data=data, ax=ax1)
plt.title("Interaction between numeric/continuous independent variables against dependent variable ")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


**Only transactionAmount shows a significant interaction with a transaction being fraudulent. Higher transaction amount are associated with fraudulent transactions. Hence only the transactionAmount will be considered from the continuous variables as a feature for the model**



In [ ]:
# select only columns of interest
data = data[['acqCountry',
            'merchantCountryCode',
            'posEntryMode',
            'posConditionCode',
            'merchantCategoryCode',
            'transactionType',
            'cardPresent',
            'transactionMonth',
            'transactionDayOfWeek',
            'transactionHour',
            'matchingCVV','transactionAmount',
            'isFraud']]

### Data Preprocessing
- preprocessing such as categorical encoding and ensuring all columns are of the numeric datatype
- Handling imbalanced in the dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder

In [ ]:
data.info() # check the data info again

In [ ]:
data.head()

In [ ]:
# create a copy
encoded_df = data.copy()
# create a label encoder object
label_encoder = LabelEncoder()

# fit and transform the data
cols_to_encode = ['acqCountry', 'merchantCountryCode', 'merchantCategoryCode', 
                  'transactionType', 'transactionMonth', 'transactionDayOfWeek']

for col in cols_to_encode:
    encoded_df[col] = label_encoder.fit_transform(encoded_df[col])
    # show the representation (check how the labelencoder has encoded the values)
    print(dict(zip(label_encoder.inverse_transform(encoded_df[col].unique()), encoded_df[col].unique())))

# make Boolean values in the data to 1's and 0's
encoded_df.replace({False: 0, True: 1}, inplace=True)

# convert all values to numeric
encoded_df = encoded_df.apply(pd.to_numeric, errors='coerce')

In [ ]:
# check all columns are numeric
encoded_df.info()

In [ ]:
# assign independent and dependent variable
X = encoded_df.drop(columns=["isFraud"])
y = encoded_df.isFraud

In [ ]:
# data partitioning
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) # shape

In [ ]:
# check how the label classes have been distributed among the sets
fig, axs = plt.subplots(1,2,figsize=(15,5)) 
# Count plot for training set
sns.countplot(x=y_train, ax=axs[0])
axs[0].set_title('Distribution of training data')
axs[0].set_xlabel('Classes')
# Count plot for test set
sns.countplot(x=y_test, ax=axs[1])
axs[1].set_title('Distribution of test data')
axs[1].set_xlabel('Classes')

print("training classes\n", pd.Series(y_train).value_counts())
print("test classes\n", pd.Series(y_test).value_counts())

In [ ]:
# apply random undersampling to the training set
from imblearn.under_sampling import RandomUnderSampler

undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train, y_train)

In [ ]:
print("train classes\n", y_resampled.value_counts())
sns.countplot(x=y_resampled)
plt.title('Distribution of classes in the  data')
plt.xlabel('Classes')
plt.show()

**9. Scikit-Learn Pipeline**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.pipeline import Pipeline

In [ ]:
pipeline_lr  = Pipeline([('scalar1',StandardScaler()),
                         ('lr_classifier',LogisticRegression())])

pipeline_knn = Pipeline([('scalar2',StandardScaler()),
                          ('knn_classifier',KNeighborsClassifier())])

pipeline_svc = Pipeline([('scalar3',StandardScaler()),
                         ('svc_classifier',SVC())])

pipeline_dt = Pipeline([('dt_classifier',DecisionTreeClassifier())])
pipeline_rf = Pipeline([('rf_classifier',RandomForestClassifier(max_depth=3))])
pipeline_gbc = Pipeline([('gbc_classifier',GradientBoostingClassifier())])

In [ ]:
pipelines = [pipeline_lr,
            pipeline_knn,
            pipeline_svc,
            pipeline_dt,
            pipeline_rf,
            pipeline_gbc]

In [ ]:
pipelines

In [ ]:
for pipe in pipelines:
    pipe.fit(X_train,y_train)

In [ ]:
pipe_dict = {0:'LR',
             1:'KNN',
             2:'SVC',
             3:'DT',
             4: 'RF',
             5: 'GBC'}

In [ ]:
pipe_dict

In [ ]:
for i,model in enumerate(pipelines):
    print("{} Test Accuracy:{}".format(pipe_dict[i],model.score(X_test,y_test)*100))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# create a Random Forest classifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)
# train the model on the resampled training data
random_forest.fit(X_resampled, y_resampled)

In [ ]:
#random_forest =RandomForestClassifier(max_depth=3)

In [ ]:
# evaluation
from sklearn import metrics
y_pred = random_forest.predict(X_test)

f1_score = metrics.f1_score(y_test,y_pred)
class_report = metrics.classification_report(y_test,y_pred)
conf_mat = metrics.confusion_matrix(y_test,y_pred) 

print("F1 score: %.3f"%(f1_score))
print("Classification report: \n",class_report)

fig, ax = plot_confusion_matrix(conf_mat=conf_mat,
                                show_absolute=True,
                                show_normed=True,
                                colorbar=True)
plt.yticks([0, 1], ['False', 'True'])
plt.xticks([0, 1], ['False', 'True'])   
plt.show()


In [ ]:
# feature importance (plot graph of feature importances for better visualization)
feat_importances = pd.Series(random_forest.feature_importances_, index=X_resampled.columns)
(100*feat_importances).nsmallest(44).plot(kind='barh', figsize=(12,12))
plt.grid()
plt.title("Feature Importance")
plt.xlabel("Importance (%)")
plt.ylabel("Feature")
plt.show()

If I have more time:
- I will do more feature engineering and interaction to see they can help improve the model e.g getting the length of hours between the last address changed date and the transaction date.
- include other features that may capture the context of the transaction, such as:
    - Whether the transaction amount is close to the credit limit
    - Whether the transaction amount is significantly different from the average transaction amount for that account
- hyperparameter tuning/optimization and also compare more models.
- I will optimize the code and create pipeline for easy deployment

**Prediction on New Data**

In [ ]:
# new_data2 = pd.DataFrame(new_data.values, columns=X_resampled.columns)

# new_data2

In [ ]:
new_data = pd.DataFrame({
    'acqCountry':33.6,
     'merchantCountryCode':50,
    'posEntryMode':33.6,
    'posConditionCode':0.627,
    'merchantCategoryCode':35.0,
    'transactionType':50,
    'cardPresent':50,
    'transactionMonth':72.0,
    'transactionDayOfWeek':79.799479,
    'transactionHour':148.0,
    'matchingCVV':0.627,
    'transactionAmount':6
  
},index=[0])


In [ ]:
new_data

In [ ]:
p = random_forest.predict(new_data)
#p = rf.predict(new_data)

In [ ]:
p[0]

In [ ]:
if p[0] == 0:
    print('Not Fraud')
else:
    print('Fraud')

**18. Save Model Using Joblib**

In [ ]:
import joblib

In [ ]:
joblib.dump(random_forest,'model_Capital_Bank')

In [ ]:
model = joblib.load('model_Capital_Bank')

In [ ]:
model.predict(new_data)

**GUI**

In [ ]:
from tkinter import *
import joblib

In [ ]:
from tkinter import *
import joblib
import numpy as np
from sklearn import *
def show_entry_fields():
    p1=float(e1.get())
    p2=float(e2.get())
    p3=float(e3.get())
    p4=float(e4.get())
    p5=float(e5.get())
    p6=float(e6.get())
    p7=float(e7.get())
    p8=float(e8.get())
    p9=float(e9.get())
    p10=float(e10.get())
    p11=float(e11.get())
    p12=float(e12.get())

    
   
    model = joblib.load('model_Capital_Bank')
    result=model.predict([[p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12]])
    
    if result == 0:
        Label(master, text="Not Fraud").grid(row=31)
    else:
        Label(master, text="Fraud").grid(row=31)
    
    
master = Tk()
master.title("Capital One Bank Fraud Detection Using Machine Learning")


label = Label(master, text = "Capital One Bank Fraud Detection Using Machine Learning"
                          , bg = "black", fg = "white"). \
                               grid(row=0,columnspan=2)


Label(master, text="Enter Value of acqCountry").grid(row=1)
Label(master, text="Enter Value of merchantCountryCode").grid(row=2)
Label(master, text="Enter Value of posEntryMode").grid(row=3)
Label(master, text="Enter Value of posConditionCode").grid(row=4)
Label(master, text="Enter Value of merchantCategoryCode").grid(row=5)
Label(master, text="Enter Value of transactionType").grid(row=6)
Label(master, text="Enter Value of cardPresent").grid(row=7)
Label(master, text="Enter Value of transactionMonth").grid(row=8)
Label(master, text="Enter Value of transactionDayOfWeek").grid(row=9)
Label(master, text="Enter Value of transactionHour").grid(row=10)
Label(master, text="Enter Value of matchingCVV").grid(row=11)
Label(master, text="Enter Value of transactionAmount").grid(row=12)




e1 = Entry(master)
e2 = Entry(master)
e3 = Entry(master)
e4 = Entry(master)
e5 = Entry(master)
e6 = Entry(master)
e7 = Entry(master)
e8 = Entry(master)
e9 = Entry(master)
e10 = Entry(master)
e11 = Entry(master)
e12 = Entry(master)

e1.grid(row=1, column=1)
e2.grid(row=2, column=1)
e3.grid(row=3, column=1)
e4.grid(row=4, column=1)
e5.grid(row=5, column=1)
e6.grid(row=6, column=1)
e7.grid(row=7, column=1)
e8.grid(row=8, column=1)
e9.grid(row=9, column=1)
e10.grid(row=10, column=1)
e11.grid(row=11, column=1)
e12.grid(row=12, column=1)


Button(master, text='Predict', command=show_entry_fields).grid()

mainloop()